Описание данных
   * children — количество детей в семье
   * days_employed — общий трудовой стаж в днях
   * dob_years — возраст клиента в годах
   * education — уровень образования клиента
   * education_id — идентификатор уровня образования
   * family_status — семейное положение
   * family_status_id — идентификатор семейного положения
   * gender — пол клиента
   * income_type — тип занятости
   * debt — имел ли задолженность по возврату кредитов
   * total_income — ежемесячный доход
   * purpose — цель получения кредита

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Здорово, когда есть вступление в сам проект, каждый сможет быстрее разобраться в концепции / деталях работы. 

</div>

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [ ]:
import pandas as pd
try:
    df = pd.read_csv('/datasets/data.csv')
    df.info()
except:
    print('ошибка')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Таблица состоит из 12 колонок с данными трех типов: float64(2), int64(5), object(5).

Колличество строк(19351 или 21525) в колонках не совпадает, значит есть пропущенные значения или ошибки

Так же перед любой обработкой кода считаю важным взглянуть на вид данных:

In [ ]:
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Из таблицы можно сразу заметить следующие моменты:**

   * отсутствующее значение в строке 12 столбцах days_employed и total_income;
   * отрицательный трудовой стаж в столбце days_employed;
   * беспорядок, связанный с регистром в столбце education;
   * столбец purpose с большим колличеством произвольны символов, где можно найти множестово опечаток;
   * не совсем понятный столбец income_type (выглядит обрезанным, однако предполагаю, что это не так важно для данной задачи). 
    
**Рассмотрим столбцы подробнее:** (оставила только интересные случаи)

In [ ]:
print(df['children'].value_counts())

# в столбце children есть две ошибочные позиции:
#    20 - очевидно, что значение должно = 2, а 0 лишь опечатка(или наобоот - опечаткой является 2)
#    -1 - скорее всего "-" тоже опечатка, а значение = 1

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


In [ ]:
print(df['dob_years'].value_counts())
# есть 101 значение "0", явно не соответствующих реальному возрасту

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


In [ ]:
print(df['education'].value_counts())
# много дублей

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


In [ ]:
print(df['gender'].value_counts())
# Отсутствующих значений не обноружено, но есть значение XNA

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


In [ ]:
print(df['income_type'].value_counts())
# Отсутствующих значений не обноружено

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64


In [ ]:
print(df['purpose'].value_counts())
# есть сложные дубли, такие как "свадьба" и "сыграть свадьбу"

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

## Шаг 2. Предобработка данных

### Обработка пропусков

Заменять NaN на 0 или на среднее нельзя, так как это сильно изменит картину.
Предлагаю заменить пропущенные значения в столбце total_income на медиану, а в столбце days_employed на среднее значение.
Заменять такие значения разумно в рамках среднего или медиального значений по группе income_type 

In [ ]:
print(df.isna().sum()) 
# Hезультат показывает, что в таблице есть два значения с пропущенными значениями - days_employed и total_income.
# Так же понятно, что таких строк в таблице 2174.

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [ ]:
# Проверим, что все значения NaN  в стобце days_employed соответствуют значению NaN в столбце total_income
list_of_nan = df[df['days_employed'] == 0]
print(len(list_of_nan[list_of_nan['total_income'] != 0]))

0


In [ ]:
#  Расчет медианы для столбца total_income, усетом группировки по столбцу income_type
total_income_median = df.groupby('income_type')['total_income'].median() 
df['total_income'] = df['total_income'].fillna(0)

#  Расчет медианы для столбца total_income, усетом группировки по столбцу income_type
df['days_employed'] = abs(df['days_employed'])                           # тк стаж не может быть отрицательным
days_employed_count = df.groupby('income_type')['days_employed'].count() 
days_employed_sum = df.groupby('income_type')['days_employed'].sum()
days_employed_mean = days_employed_sum / days_employed_count
df['days_employed'] = df['days_employed'].fillna(0)

In [ ]:
# Заменяем нулевые значения в столбцах days_employed и total_income. Очень хотела написать цикл, но попытки ни к чему не 
# привели (кажется датафрейм не дружит с циклами). Ниже представлены только те категории income_type, в которых есть пропуски.
list_of_nan.loc[list_of_nan['income_type'] == 'сотрудник', 'total_income'] = total_income_median['сотрудник']
list_of_nan.loc[list_of_nan['income_type'] == 'сотрудник', 'days_employed'] = days_employed_mean['сотрудник']

list_of_nan.loc[list_of_nan['income_type'] == 'компаньон', 'total_income'] = total_income_median['компаньон']
list_of_nan.loc[list_of_nan['income_type'] == 'компаньон', 'days_employed'] = days_employed_mean['компаньон']

list_of_nan.loc[list_of_nan['income_type'] == 'пенсионер', 'total_income'] = total_income_median['пенсионер']
list_of_nan.loc[list_of_nan['income_type'] == 'пенсионер', 'days_employed'] = days_employed_mean['пенсионер']

list_of_nan.loc[list_of_nan['income_type'] == 'госслужащий', 'total_income'] = total_income_median['госслужащий']
list_of_nan.loc[list_of_nan['income_type'] == 'госслужащий', 'days_employed'] = days_employed_mean['госслужащий']

list_of_nan.loc[list_of_nan['income_type'] == 'предприниматель', 'total_income'] = total_income_median['предприниматель']
list_of_nan.loc[list_of_nan['income_type'] == 'предприниматель', 'days_employed'] = days_employed_mean['предприниматель']

In [ ]:
# проверка
print(len(list_of_nan[list_of_nan['days_employed'] == 0]), len(list_of_nan[list_of_nan['total_income'] == 0]))

0 0


**Вывод**

Отсутствующие значения успешно заменены на наиболее подходящие с учетом группы занятости.

Так же смущает величина стажа (люди столько не живут): рассмотрим пример из строки 4, где стаж составляет 340266 дней, с учетом работы без выходных мы получаем 932 года, многовато...

Как пересчитать такие данные не знаю, но есть варианты: взять среднее во каким-нибудь группам (возраст, семейное положение или должность) или удалить строки, если таких значений немного.

И в идеале стоит заменить нулевые значения возраста в колонке dob_years (однако таких значений совсем немного - 0,5%)

### Замена типа данных

In [ ]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

**Вывод**

Из изученных методов на курсе, astype подходит лучше всего, так как преобразует значения столбца в нудный мне тип данных - int
В таблицезамена типа данных была произведена только в двух столбцах: total_income и days_employed; так как их формат был неудобенранее ( нецелочисленные значения с 6 числами после запятой)

### Обработка дубликатов

Ранее выяснилось, что дубли встречаются в двух столбцах: education и purpose. 
Обработать столбец education проще всего ручным удалением дублей, изменив регистр на однотипный. 
А вот для обработки второго столбца лучше всего подойдет леммитизаяция, так как дублирование там связано с описанием одной 
тематики разными выражениями.

In [ ]:
df['education'] = df['education'].str.lower()
# проверка: выведем все уникальные значения
print(df['education'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


**Вывод**

Объединение столбцов по категориям не имеет смысла, так как мы рассматриваем конкретных индивидумов.
Проверка показала, что понижение регистра достаточно, для корректтной разбивки по категориям.

### Лемматизация

In [ ]:
from pymystem3 import Mystem
m = Mystem() 

immovables = m.lemmatize(df['purpose'][10])[2]
education = m.lemmatize(df['purpose'][3])[2]
wedding = m.lemmatize(df['purpose'][4])[2]
car = m.lemmatize(df['purpose'][1])[2]

In [ ]:
for el in range(0, len(df)):
    if (df['purpose'][el] == 'свадьба' or df['purpose'][el] == 'на проведение свадьбы ' or
        df['purpose'][el] == 'сыграть свадьбу'):
            df['purpose'][el] = wedding
            
    elif (df['purpose'][el] == 'на покупку своего автомобиля' or df['purpose'][el] == 'автомобиль' or
        df['purpose'][el] == 'сделка с подержанным автомобилем' or df['purpose'][el] == 'свой автомобиль' or
        df['purpose'][el] == 'на покупку подержанного автомобиля' or df['purpose'][el] == 'автомобили' or
        df['purpose'][el] == 'на покупку автомобиля' or df['purpose'][el] == 'приобретение автомобиля' or
        df['purpose'][el] == 'сделка с автомобилем'):      
            df['purpose'][el] = car
            
    elif (df['purpose'][el] == 'заняться высшим образованием' or df['purpose'][el] == 'дополнительное образование' or
        df['purpose'][el] == 'высшее образование' or df['purpose'][el] == 'получение образования' or
        df['purpose'][el] == 'получение дополнительного образования' or df['purpose'][el] == 'образование' or    
        df['purpose'][el] == 'профильное образование' or df['purpose'][el] == 'получение высшего образования' or
        df['purpose'][el] == 'заняться образованием'):
            df['purpose'][el] = education
    
    else:
        df['purpose'][el] = immovables

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher

In [ ]:
# проверка
df['purpose'].value_counts()

недвижимость    11617
автомобиль       4315
образование      4022
свадьба          1571
Name: purpose, dtype: int64

**Вывод**

Понимаю, что алгоритм совсем не оптимален, однако способа лучше реализовать у меня не вышло. 
Выбрала метод lemmatize, так как он позволяет разделять датасет по отдельным словам.
Как показала проверка, данные успешно разделены на 4 запланированные группы.
Алгоритм леммитизации прост: визуально просматриваем группы, полученные на первом шаге и разделяем их на 4 группы:
недвижимостьб автомобиль, образование и свадьба. Далее присваеваем каждой группе необходимые значения. 
Я не стала делить частную недвижимость с комерческой, так как не совсем понятна грань между ними.

### Категоризация данных

In [ ]:
# создадим таблицу для удобства ответа на первый вопрос
first_question=df[['children','debt']]
first_question

,children,debt
0,1,0
1,1,0
2,0,0
3,3,0
4,0,0
...,...,...
21520,1,0
21521,0,0
21522,1,1
21523,3,1


In [ ]:
# создадим таблицу для удобства ответа на второй вопрос
second_question=df[['family_status_id','debt']]
second_question

,family_status_id,debt
0,0,0
1,0,0
2,0,0
3,0,0
4,1,0
...,...,...
21520,1,0
21521,0,0
21522,1,1
21523,0,1


In [ ]:
# создадим таблицу для удобства ответа на третий вопрос
third_question=df[['total_income','debt']]
third_question

,total_income,debt
0,253875,0
1,112080,0
2,145885,0
3,267628,0
4,158616,0
...,...,...
21520,224791,0
21521,155999,0
21522,89672,1
21523,244093,1


In [ ]:
# создадим таблицу для удобства ответа на четвертый вопрос
fourth_question=df[['purpose','debt']]
fourth_question

,purpose,debt
0,недвижимость,0
1,автомобиль,0
2,недвижимость,0
3,образование,0
4,свадьба,0
...,...,...
21520,недвижимость,0
21521,автомобиль,0
21522,недвижимость,1
21523,автомобиль,1


**Вывод**

## Шаг 3. Ответьте на вопросы

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [ ]:
# Для начала посмотрим размер выборки во каждой группе children
first_question.groupby('children').count()

,debt
children,
-1,47
0,14149
1,4818
2,2055
3,330
4,41
5,9
20,76


In [ ]:
# выборки явно отличаются по размеру, поэтому найдем процент просрочек по выплотам, по отношению к общему количеству кредитов
question1 = first_question.groupby('children').sum() * 100 / first_question.groupby('children').count()
question1

,debt
children,
-1,2.127660
0,7.512898
1,9.215442
2,9.440389
3,8.181818
4,9.756098
5,0.000000
20,10.526316


**Вывод**

по первому действию видно, что наиболее значимые группы - 0, 1, 2 детей. Если рассматривать только эти группы, то процент невозврата задолженности в срок увеличивается с ростом числа детей. 

ОДнако стоит учесть, что этот рост слишком мал, что бы делать такие серьезные выводы, тем более результаты по семьям с 3 и 4 
детьми идеют очень сходий процент. 

Я считаю, что зависимости между колличеством детей и возвратом кредита в срок нет.

Стоит пояснить про остальные значения:

    * -1 и 20 - выборка составляет 47 и 76 человек соответственно, слишком мало, что бы учитывать эти данные
    * 5 - выборка из 9 человек, которые вернули кредит во время. Однако выборка тоже слишком мала, что бы учитывать 
    этот результат

- **Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [ ]:
# ситуация схожая, поэтому повторим практику с процентами
second_question.groupby('family_status_id').count()

,debt
family_status_id,
0,12380
1,4177
2,960
3,1195
4,2813


In [ ]:
question2 = second_question.groupby('family_status_id').sum() * 100 / second_question.groupby('family_status_id').count()
question2

,debt
family_status_id,
0,7.520194
1,9.288963
2,6.562500
3,7.112971
4,9.740491


**Вывод**

И снова не видно никакой четкой зависимости. Ответ: нет

- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [ ]:
# Добавим флаги по доходам, что б было удобнее сортировать
def total_income_function(income):
    if income <= 50000:
        return 1 
    elif income <= 100000:
        return 2
    elif income <= 150000:
        return 3
    elif income <= 200000:
        return 4
    elif income > 200000:
        return 5

In [ ]:
# применим функцию total_income_function
third_question.loc[:,'total_income_flag'] = third_question.loc[:, 'total_income'].apply(total_income_function)

# вырежем из таблицы столбец доходов
third_question = third_question[['total_income_flag','debt']]

# и посмотрим на распреление выборки
third_question.groupby('total_income_flag').count()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,debt
total_income_flag,
1,2546
2,4091
3,5704
4,4118
5,5066


In [ ]:
question3 = third_question.groupby('total_income_flag').sum() * 100 / third_question.groupby('total_income_flag').count()
question3

,debt
total_income_flag,
1,7.580518
2,8.090931
3,8.607994
4,8.936377
5,7.066719


**Вывод**

Ответ: нет, это видно из результатов, процент просрочки по кредитам составляет примерно 8%

- **Как разные цели кредита влияют на его возврат в срок?**

In [ ]:
question4 = fourth_question.groupby('purpose').sum() * 100 / fourth_question.groupby('purpose').count()
question4

,debt
purpose,
автомобиль,9.339513
недвижимость,7.282431
образование,9.199403
свадьба,7.765754


**Вывод**

градация присутствует (с ростом процента просрочки) :
    * кредит за недвижимость (т.к. она в залоге)
    * кредит за свадьбу (т.к. он обычно не велик)
    * кредит за образование
    * кредит за автомобиль

## Шаг 4. Общий вывод

Работа оказалась достаточно творческой в плане очистки данных. Вывод по задаче достаточно логичен - из всех категорий возвращаемость кредитов в срок зависит только от цели кредита.